<div style="border:1px solid black; padding:10px 10px;">
    <strong>CIVIL-321 "Modélisation Numérique des Solides et Structures"</strong><br/><br/>
    <span style="text-decoration:underline;font-weight:bold;">Comment utiliser ce Jupyter Notebook?
    </span><br/><br/>
    Ce <strong>Notebook</strong> est constitué de cellules de texte et de cellule de code. Les cellules de codes doivent être  <strong>executées</strong> pour voir le résultat du programme. Certaines cellules doivent être remplies par vos soins. Pour exécuter une cellule, cliquez dessus simplement et ensuite cliquez sur le bouton "play" (<span style="font: bold 12px/30px Arial, serif;">&#9658;</span>) dans la barre de menu au dessus du notebook. Vous pouvez aussi taper la combinaison de touches <code>shift + enter</code>. Il est important d'éxécuter les cellules de code en respectant leur ordre d'arrivée dans le notebook.
</div>

On vous encourage à poser vos questions et donner votre feedback sur ce notebook sur la plateforme ED Discussion du cours accessible en cliquant sur ce bouton:
 
 
 
<div class="container" >
        <a href="https://edstem.org/eu/courses/409/discussion?category=Exercices">
            <button class="btn btn-primary btn-lg">Ed Discussion</button>
        </a>
</div>

In [ ]:
import numpy as np
from plot import *

# Barre modélisée comme un milieu continu

Dans cette série, nous allons résoudre un problème de milieu continu (une poutre console) et manipuler des maillages automatiquement générés par Gmsh, un mailleur open-source.

## Poutre bi-dimensionnelle
*Géométrie et conditions limites.*
La poutre console montrée sur la Figure a une longeur $L_{x} = 500$ mm, une hauteur $L_{y} = 100$ mm et épaisseur $t = 10$ mm.
On encastre complètement la partie gauche de la poutre, et on charge le coin supérieur droit avec une force $P = 100$ N. Les constantes d'élasticité sont $E = 210$ GPa et $\nu = 0.3$.
![](Images/fig1.svg)
La structure est maillée avec des éléments triangulaires linéaires (déformation constante). Le maillage est structuré selon deux directions, comme le montrent les figure ci dessous, avec $n_{x}=15$ et $n_{y}=3$ les nombres d'éléments selon $x$ et $y$ respectivement:

![](Images/fig2.svg)
*Numérotation des noeuds pour $n_{x} = 15$ et $n_{y}$ = 3.*

1. Déterminer les espacements $l_{x}$ et $l_y$ entre deux noeuds consécutifs, ainsi que les positions des noeuds 1, 50 et 63 montrés sur la figure ci-dessus, numérotés comme indiqué.

---

 **Solution:**

 ---




$$\begin{align*}
n_x &= 15\\
n_y &= 3\\
l_{x} &= \frac{L_{x}}{n_{x}} = 33.333 \mbox{ mm}\\
l_{y} &= \frac{L_{y}}{n_{y}} = 33.333 \mbox{ mm}\\
\end{align*}$$

$$\begin{array}{ccc}
  Noeud & x (mm) & y (mm) \\
  \hline
  0 & 0\cdot l_{x}=33.33 & 0\cdot l_{y}=0.00 \\
  1 & 1\cdot l_{x}=33.33 & 0\cdot l_{y}=0.00 \\
  16 & 0\cdot l_{x}=0.00 & 1\cdot l_{y}=33.33 \\
  50 & 2\cdot l_{x}=66.67 & n_{y}\cdot l_{y}=100.00 \\
  & ... & \\
  63 & n_{x}\cdot l_{x}=500.00 & n_{y}\cdot l_{y}=100.00 \\
\end{array}$$

2. Déterminer les coordonnées du noeud $n$ pour $n_x$ et $n_y$ arbitraires.

---

 **Solution:**

 ---



Les noeuds sont répartis sur une grille de taille ($n_x+1$, $n_y+1$). Le noeud en position ($a$,$b$) a pour coordonnées ($a\cdot l_{x}$,$b\cdot l_{y}$). Il reste à déterminer la position ($a$,$b$) en fonction de son index global $n$:

$$\begin{align*}
a &= n \, \% \, (n_x+1)\\
b &= E\big[ n/(n_x+1)\big]\\
\end{align*}$$

avec $E\big[x\big]$ la partie entière de $x$.

2. Écrire le code Python calculant la matrice des coordonnées pour des $n_x$ et $n_y$ arbitraires (cette matrice a deux colonnes, pour les positions $x$ et $y$). 

*indice: il est intéressant de trouver la relation entre les coordonnées dans la grille $i, j$ d'un noeud et son index global $n$ . Par exemple le noeud $18$ a pour coordonnées $(2, 1)$*

In [ ]:
###########
# Solution:
##########


def calculerCoordonneesNoeuds(Lx, Ly, nx, ny):
    dx = Lx / nx;
    dy = Ly / ny;

    coordonnees = np.zeros(((nx+1)*(ny+1), 2))
    for i in range(nx+1):
        for j in range(ny+1):
            n = i + j*(nx+1)
            coordonnees[n, :] = [i*dx, j*dy]
    return coordonnees

coords = calculerCoordonneesNoeuds(500, 100, 15, 3)
Matrix(coords)

## connectivité

![](Images/fig2.svg)

![](Images/fig3.svg)
*Numérotation des éléments pour $n_{x} = 15$ et $n_{y}$ = 3.*

3. Déterminer la connectivité des éléments 0, 30 et 45 pour la numérotation indiquée.

---

 **Solution:**

 ---




- Numérotons les éléments dans le sens trigonométrique, en commençant du coin gauche. Les noeuds locaux ($i$, $j$, $m$) de l'élément 1 sont 0, 1 et 17.

$$\begin{array}{cccc}
 Élement & i & j & m \\
 \hline
 0 & 0 & 1 & 17 \\
 30 & 32 & 33 & 49 \\
 45 & 0 & 17 & 16 \\
 \end{array}$$

![](Images/fig4.svg)

3. Déterminer la connectivité d'un élément $e$ pour $n_x$ et $n_y$ arbitraires.

---

 **Solution:**

 ---



- On considère les éléments comme des cellules positionnés sur une grille de taille $(n_x, n_y)$. Sur une cellule $(i, j)$ de la grille, on peut placer deux éléments triangulaires, que l'on numérote comme sur la figure.

  Pour chaque cellule $(i, j)$, on a quatre noeuds qui forment des coins. On numérote ces noeuds dans le sens trigonométrique de 0 à 3, en partant du coin en bas à gauche. Nous avons donc la numérotation globale des coins de la cellule $(i, j)$ donnée par:

$$\begin{eqnarray}
n_1 & = &j n_x + i\\
n_2 & = &n_1 + 1\\
n_3 & = &n_1 + n_x + 1 \\
n_4 & = &n_3 + 1\\
\end{eqnarray}$$

On peut à présent former la connectivité des éléments.

$$\begin{array}{cccc}
Élement & i & j & m \\
\hline
e_{A} & n_1 & n_2 & n_4 \\
e_{B} & n_1 & n_4 & n_3 \\
\end{array}$$

En ce qui concerne la matrice des connectivités complète, elle a $n_{elem}=n_{x}\cdot n_{y}\cdot 2$ lignes et 3 colonnes.
Les $n_{x}\cdot n_{y}$ premières lignes sont occupées par les éléments $A$, les $n_{x}\cdot n_{y}$ lignes suivantes sont des éléments de type $B$.

4. Implémentez le code Python construisant la matrice de connectivité. Controlez votre mesh avec la routine `plotMesh(coords, conn)`

In [ ]:
###########
# Solution:
##########
 

# Calcul de la connectivité
def calculerConnectivite(Lx, Ly, nx, ny):
    connectivite = np.zeros((2*nx*ny, 3), dtype=int)
    for i in range(nx):
        for j in range(ny):
            n  = j*nx + i      # Numéro de l'élément
            n1 = j*(nx+1) + i  # Noeud bas-gauche
            n2 = n1 + 1        # Noeud bas-droite
            n3 = n1 + (nx+1)   # Noeud haut-gauche
            n4 = n3 + 1        # Noeud haut-droite
        
            # On numérote les noeuds dans le sens trigonométrique
            connectivite[n, :] = [n1, n2, n4];       # Triangle bas
            connectivite[n+nx*ny, :] = [n1, n4, n3]  # Triangle haut
    return connectivite

conn = calculerConnectivite(500, 100, 15, 3)

plotMesh(coords, conn)
Matrix(conn)

5. Implémentez le code Python construisant la matrice des numéros d'équation.

In [ ]:
###########
# Solution:
##########


def calculerNumerosEquations(connectivite):

    n_elem  = connectivite.shape[0]
    n_nodes_per_elem = connectivite.shape[1]
    numEq = np.zeros((n_elem, 2*n_nodes_per_elem), dtype=int)
    for e in range(n_elem):
        for i in range(n_nodes_per_elem):
            numEq[e, 2*i]   = 2*connectivite[e, i];
            numEq[e, 2*i+1] = 2*connectivite[e, i]+1;
    return numEq
            
numEq = calculerNumerosEquations(conn)
Matrix(numEq)

## Rigidité

5. Implémenter une fonction calculant la matrice de rigidité locale $K^e$ d'un élément $e$.

In [ ]:
def calculerD(E, nu, contraintes_planes=True):
    # remplir ici
    pass


def calculerB(connectivite_element, coordonnees):
    # remplir ici
    B = None # B matrix
    A = None # area of current triangular element
    return B, A


def calculerMatriceRigiditeLocale(E, nu, t, connectivite_element, coordonnees):
    D = calculerD(E, nu)

    # On calcule B et l'aire du triangle pour l'intégrale
    [B, A] = calculerB(connectivite_element, coordonnees)

    #remplir ici
    pass
    #return K_locale

E = 210e3
t = 10.
nu = 0.3

# plotting local matrix of element 0
Kloc = calculerMatriceRigiditeLocale(E, nu, t, conn[0], coords)
Kloc

In [ ]:
###########
# Solution:
##########


def calculerD(E, nu, contraintes_planes=True):

    # On fait des contraintes planes par défaut
    if contraintes_planes:
        D = (E/(1-nu**2))* np.array(
            [[1,  nu, 0],
             [nu, 1,  0],
             [0,  0,  (1-nu)/2]])
    else:
        raise RuntimeError('Déformations planes à implémenter!')
    return D


def calculerB(connectivite_element, coordonnees):
    noeuds = coordonnees[connectivite_element.ravel(), :]

    beta = zeros(3, 1);
    gamma = zeros(3, 1);

    gamma[0] = noeuds[2, 0] - noeuds[1, 0]
    gamma[1] = noeuds[0, 0] - noeuds[2, 0]
    gamma[2] = noeuds[1, 0] - noeuds[0, 0]

    beta[0] = noeuds[1, 1] - noeuds[2, 1]
    beta[1] = noeuds[2, 1] - noeuds[0, 1]
    beta[2] = noeuds[0, 1] - noeuds[1, 1]

    A = 0.5*(noeuds[0, 0]*beta[0]+noeuds[1, 0]*beta[1]+noeuds[2, 0]*beta[2])

    B = 1/(2*A) * np. array([[beta[0],  0,        beta[1],  0,        beta[2],  0],       
                             [0,        gamma[0], 0,        gamma[1], 0,        gamma[2]],
                             [gamma[0], beta[0],  gamma[1], beta[1],  gamma[2], beta[2]]])
    return B, A


def calculerMatriceRigiditeLocale(E, nu, t, connectivite_element, coordonnees):
    D = calculerD(E, nu)

    # On calcule B et l'aire du triangle pour l'intégrale
    [B, A] = calculerB(connectivite_element, coordonnees)

    # Comme B est constante, on intègre facilement
    K_locale = A * t * B.T @ D @ B
    return K_locale

E = 210e3
t = 10.
nu = 0.3

# plotting local matrix of element 0
Kloc = calculerMatriceRigiditeLocale(E, nu, t, conn[0], coords)
Matrix(Kloc)

6.Adapter la fonction `assemblerMatriceRigidite()` pour assembler la matrice de rigidité globale du système *K*.

In [ ]:
# Cette routine d'assemblage est celle de l'exercice 02_BarresTreillis/02_Exercice.ipynb
# il faut la modifier

def AssemblerMatriceRigidite(k_vec, positions, conn, nb_elem, nb_ddl_tot, eqn_elem):
    K = np.zeros((nb_ddl_tot, nb_ddl_tot))
    
    for e in range(nb_elem):
        # Construction de la matrice de rigidité locale
        Kl = calculerMatriceRigiditeLocale(k_vec[e])
        # Construction de la matrice de rotation
        R = CalculerMatriceRotation(e, positions, conn)
        # Rotation de la matrice dans le système global de coordonnée
        Kg = R@Kl@R.T
        
        # Assemblage dans la matrice de rigidité du système à l'aide des
        idx = eqn_elem[e, :]
        for i, gi in enumerate(idx):
            for j, gj in enumerate(idx):
                K[gi, gj] += Kg[i, j]
    return K

# un routine de cette forme est espérée:

def assemblerMatriceRigidite(E, nu, t, connectivite, coordonnees):
    # remplir ici
    pass

In [ ]:
###########
# Solution:
##########


def assemblerMatriceRigidite(E, nu, t, connectivite, coordonnees):

    n_elem  = connectivite.shape[0]
    n_nodes = coordonnees.shape[0]
    numEq = calculerNumerosEquations(connectivite)

    K = np.zeros((n_nodes*2, n_nodes*2))
    for e in range(n_elem):
        # On récupère les degrés de liberté de l'élément e
        ddl = numEq[e, :]
        # On récupère les noeuds de l'élément e
        connectivite_element = connectivite[e, :]
        # On calcule la matrice de rigidite locale de l'élément e
        K_locale = calculerMatriceRigiditeLocale(E, nu, t, connectivite_element, coordonnees)
        # On assemble
        for i, gi in enumerate(ddl):
            for j, gj in enumerate(ddl):
                K[gi, gj] += K_locale[i, j]
    return K

K = assemblerMatriceRigidite(E, nu, t, conn, coords)

# dessine le profile des coefficients non zero
import matplotlib.pylab as plt
plt.spy(K)

6. Déterminer, pour $n_x$ et $n_y$ arbitraires, quels sont les degrés de liberté bloqués par la condition limite d'encastrement ? Implémenter une fonction `calculerBlocages()` qui retourne un vecteur de taille $n_{nodes} \times 2$ qui contient `True` si le degré de liberté est bloqué et `False` sinon.

In [ ]:
###########
# Solution:
##########


def calculerBlocages(nx, ny):

    # On connait à l'avance le nombre de ddl bloqués
    blocages = np.zeros(((nx+1)*(ny+1), 2), dtype=bool)
    for j in range(ny+1):
        # On bloque les degrés de liberté correspondants
        blocages[0+(nx+1)*j, :] = True
        
    return blocages.ravel()

blocages = calculerBlocages(15, 3)
libres = np.logical_not(blocages)
libres

7. Calculer la valeur des déplacements nodaux en utilisant $n_x = 15$ et $n_y = 3$ et $P = 100$. Vous    pouvez visualiser votre résultat en utilisant la routine `plotMesh(coords, conn, u)`

In [ ]:
nx = 15
ny = 3
n_nodes = (nx+1)*(ny+1)

# Vecteur déplacements
u = np.zeros(2*(nx+1)*(ny+1))

# Vecteur force
P = -100;
F = np.zeros(n_nodes*2)
F[-1] = P

# Résolution du système Ku=F avec conditions limites
u[libres] = np.linalg.solve(K[libres, :][:, libres], F[libres])

plotMesh(coords+u.reshape(n_nodes, 2)*1000, conn, u)

8. Calculer la valeur du déplacement vertical maximum.

In [ ]:
###########
# Solution:
##########


u.reshape(n_nodes, 2)[:, 1].min()

9.Calculer la valeur théorique en utilisant la théorie des poutres de Bernoulli et comparer avec la valeur trouvée par éléments finis.

---

 **Solution:**

 ---



Avec $n_{x}=15$ et $n_{y}=3$:

\begin{align*}
\mbox{min}(v)=-1.78\cdot 10^{-2}\mbox{ mm}
\end{align*}

Valeur théorique:
\begin{align*}
\delta= \frac{PL^{3}_{x}}{3EI}=-2.381\cdot 10^{-2}\mbox{ mm}
\end{align*}

Erreur:
\begin{align*}
e = \left|\frac{\mbox{min}(v)-\delta}{\delta}\right|= 25\%
\end{align*}

8. Faire varier le nombre d'éléments dans les directions $x$ et $y$ pour obtenir une valeur du déplacement qui diffère de moins de 1\% de la valeur obtenue par la théorie des poutres. Essayer d'optimiser le nombre d'éléments.

In [ ]:
###########
# Solution:
##########


# gather in one place the total problem
def solve(nx, ny):
    E = 210e3
    t = 10.
    nu = 0.3
    coords = calculerCoordonneesNoeuds(500, 100, nx, ny)
    conn = calculerConnectivite(500, 100, nx, ny)
    K = assemblerMatriceRigidite(E, nu, t, conn, coords)
    blocages = calculerBlocages(nx, ny)
    libres = np.logical_not(blocages)
    
    n_nodes = (nx+1)*(ny+1)
    u = np.zeros(2*(nx+1)*(ny+1))
    P = -100;
    F = np.zeros(n_nodes*2)
    F[-1] = P

    u[libres] = np.linalg.solve(K[libres, :][:, libres], F[libres])
    return u, coords, conn

# compute the bernoulli solution
P = -100
L = 500
h = 100
I = t*h**3/12

bernoulli_solution = P*L**3/(3*E*I) # 1e-2 is for unit conversion

# study several mesh refinement in a loop
results = []
for k in range(1, 8):
    nx = 15*k
    ny = 3*k
    n_nodes = (nx+1)*(ny+1)
    _u, _coords, _conn = solve(nx, ny)
    v_min = _u.reshape(n_nodes, 2)[:, 1].min()
    error = (bernoulli_solution-v_min)/bernoulli_solution
    print('nx:', nx, 'ny:',ny, 'v_min:',v_min, 'bernoulli:', bernoulli_solution, 'error:', error)
    results.append((k, nx, ny, v_min, bernoulli_solution, error, _u, _coords, _conn))
    
# plot the prediction
results = np.array(results, dtype='object')
plt.plot(results[:, 1], results[:, 3], '-o', label='FE')
plt.plot(results[:, 1], results[:, 4], '--', label='bernoulli')
plt.xlabel('$n_x$')
plt.ylabel(r'$v$')
l = plt.legend()

# plot the error
# results = np.array(results)
# plt.plot(results[:, 1], results[:, 5], '-o')
# plt.xlabel('$n_x$')
# plt.ylabel(r'$Err = \left|\frac{v_{min}-\delta}{\delta}\right|$')

9. Déterminer la contrainte maximum le long de l'axe $y$ pour $x = 0$, en utilisant $n_x = 5$ et $n_y = 1$, $n_x = 15$ et $n_y = 3$, puis en utilisant les valeur de $n_x$ et $n_y$ utilisée précédemment. Comparer les valeurs avec la théorie des poutres. Commenter la convergence.

In [ ]:
###########
# Solution:
##########


# Le vecteur (en notation de Voigt) du tenseur des contraintes peut être calculé à partir du vecteur des déplacements:
# sigma = [D][B]{d}
# epsilon = [B]{d} est le vecteur (en notation de Voigt) du tenseur petites déformations. 
# La matrice B a déjà été calculée lors du calcul des rigidités locales, on peut donc s'en resservir. 
# Dans le script proposé, une fonction calcule les contraintes pour un élément donné.

# Calcul des contraintes par element

def calculerContrainteElement(E, nu, deplacements, element, connectivite, equation_num, coordonnees):
    connectivite_element = connectivite[element, :]
    B, A = calculerB(connectivite_element, coordonnees)
    D = calculerD(E, nu)
    n_equations = equation_num[element, :]
    sigma = D @ B @ deplacements[n_equations]
    return sigma

# On parcourt les éléments: pour chaque élément, on calcule le tenseur des
# contraintes et on stocke la valeur sigma_x dans un vecteur

def calculerContraintes(E, nu, deplacements, connectivite, coordonnees):
    num_element = connectivite.shape[0]
    sigma_x = np.zeros(num_element)
    equation_num = calculerNumerosEquations(connectivite)
    for e in range(num_element):
        sigma = calculerContrainteElement(E, nu, deplacements, e, connectivite, equation_num, coordonnees)
        sigma_x[e] = sigma[0]
    return sigma_x

# choose a set of parameters
P = -100
L = 500
h = 100
I = t*h**3/12

## Solution de Bernoulli de la contrainte maximale
# Calcul de la valeur théorique
sigma_bernoulli = np.abs(P) * L * h / (2*I)
print('Contrainte Bernoulli = ', sigma_bernoulli, 'MPa')

res_sigma = []
for k in range(results.shape[0]):
    res = results[k]
    _u = res[6]
    _coords = res[7]
    _conn = res[8]
    
    sigma_x = calculerContraintes(E, nu, _u, _conn, _coords)
    print('pour nx:', res[1])
    sigma_FE = max(sigma_x)
    print('Contrainte FE = ', sigma_FE, 'MPa')    
    
    # Calcul d'erreur
    erreur = abs(sigma_FE - sigma_bernoulli)/abs(sigma_bernoulli)
    print('Erreur contrainte = ', 100*erreur, '%')
    res_sigma.append((res[1], sigma_FE, sigma_bernoulli))


# plot results
res_sigma = np.array(res_sigma)
plt.plot(res_sigma[:, 0], res_sigma[:, 1], '-o', label='FE')
plt.plot(res_sigma[:, 0], res_sigma[:, 2], '--', label='bernoulli')
plt.xlabel('$n_x$')
plt.ylabel(r'$\sigma$')
l = plt.legend()

# L'erreur diminiue quand on augmente le nombre d'éléments. 
# Par contre, à partir d'un certain nombre d'éléments, 
# les contraintes calculées sont plus grandes que les contraintes théoriques. 
# Cela est dû à une singularité causée par les conditions limites: plus le maillage 
# devient fin, plus les contraintes sont grandes.

# Format MSH

Dans cet exercice, nous allons utiliser une fonction pour pouvoir charger un maillage déjà existant sur Python. 

1. Le fichier *carre.msh* contient un maillage. Ouvrez *carre.msh* (dans noto ou en téléchargant le fichier). Construisez à la main la connectivité du maillage. Vous pouvez vous aider de la documentation de Gmsh (http://gmsh.info/doc/texinfo/gmsh.html#MSH-file-format-version-2-_0028Legacy_0029).
2. Chargez le maillage avec la fonction `readMesh(carre.msh)`. Vérifier que les coordonnées et connectivité sont correctes. Affichez le maillage avec la fonction `plotMesh`.

In [ ]:
###########
# Solution:
##########


coords, conn = readMesh('carre.msh')
plotMesh(coords, conn)

3. Téléchargez et installez le mailleur open-source Gmsh (http://gmsh.info/).
4. Ouvrez *carre.msh* avec Gmsh. Pour afficher la numérotation des noeuds et éléments, allez dans *Tools $\Rightarrow$ Options $\Rightarrow$ Mesh $\Rightarrow$ Visibility* et cochez *Nodes*, *Node labels* et *Surface labels*.
5. Supposons que les noeuds de la face supérieure du carré se déplacent horizontalement de $0.1$ m (Sans réaliser de calcul par éléments finis). Affichez la déformée.

In [ ]:
###########
# Solution:
##########


u = np.zeros((4, 2))
u[2:, 0] = 0.1
plotMesh(coords+u, conn)

# Maillage avec GMSH

  Nous allons utiliser Gmsh afin de mailler la structure suivante à l'aide de triangle T6:
  
  
![](Images/A.svg)

1. A l'aide de Gmsh, nous allons d'abord préciser les contours de la géométrie étudiée (en unité [m]) avec la marche à suivre suivante:
  - Créez un paramètre *lc*, qui sera le paramètre de rafinement du maillage, à l'aide de l'option *Geometry $\Rightarrow$ Elementary entities $\Rightarrow$ Add $\Rightarrow$ Parameter*
  - Créez les points d'angle à l'aide de l'option *Geometry $\Rightarrow$ Elementary entities $\Rightarrow$ Add $\Rightarrow$ Point*. 
    Entrez les coordonnées, et mettez *lc* dans le champ *Prescribed mesh element size at point*.
  - Connectez ces points à l'aide de lignes droites en utilisant l'option *Geometry $\Rightarrow$ Elementary entities $\Rightarrow$ Add $\Rightarrow$ Straight line*.
  - A présent, utilisez ces lignes pour définir la surface de la structure (*Geometry $\Rightarrow$ Elementary entities $\Rightarrow$ Add 
    $\Rightarrow$ Plane surface*). Séléctionnez d'abord le bord extérieur, puis le bord intérieur.
  - Finalement, il faut définir la surface créée comme ``physical entity'' afin qu'elle soit maillée par Gmsh. Pour ce faire, sélectionner la surface créée à l'aide de *Geometry $\Rightarrow$ Physical groups $\Rightarrow$ Add $\Rightarrow$ Surface*.
  - Sauvegardez la géométrie créée avec le nom *solide.geo*. 

2. Ouvrez le fichier *solide.geo* à l'aide d'un éditeur de texte (par exemple le Notepad). Observez le contenu du fichier généré par Gmsh. Vous pouvez modifier le fichier et observer les changements dans Gmsh une fois le fichier ré-ouvert.
3. Afin de mailler la structure, utilisez le menu *Mesh $\Rightarrow$ 2D*. Afin d'utiliser des éléments T6, utilisez l'option *Set order 2*. Sauvegardez le maillage ainsi créé à l'aide de l'option *File $\Rightarrow$ Save Mesh* en utilisant le nom par défaut *solide.msh*. Vous pouvez modifier directement dans Gmsh la valeur du paramètre *lc* pour raffiner le maillage.
4. Vous pouvez répéter les questions Python de l'exercice précédent avec le nouveau maillage *solide.msh*..

In [ ]:
###########
# Solution:
##########


# Le fichier geo que vous devriez obtenir devrait ressembler a ca:

geo_file = """

lc = DefineNumber[ 0.7, Name "Parameters/lc" ];
Point(1) = {0, 0, 0, lc};
Point(2) = {2, 0, 0, lc};
Point(3) = {8, 0, 0, lc};
Point(4) = {10, 0, 0, lc};
Point(5) = {4, 6, 0, lc};
Point(6) = {6, 6, 0, lc};
Point(7) = {6, 7, 0, lc};
Point(8) = {4, 7, 0, lc};
Point(9) = {5, 10, 0, lc};
Point(10) = {4, 11, 0, lc};
Point(11) = {6, 11, 0, lc};
Line(1) = {1, 2};
Line(2) = {2, 5};
Line(3) = {5, 6};
Line(4) = {6, 3};
Line(5) = {3, 4};
Line(6) = {4, 11};
Line(7) = {11, 10};
Line(8) = {10, 1};
Line(9) = {8, 9};
Line(10) = {9, 7};
Line(11) = {7, 8};
Line Loop(12) = {11, 9, 10};
Line Loop(13) = {8, 1, 2, 3, 4, 5, 6, 7};
Plane Surface(14) = {12, 13};
Physical Surface(15) = {14};

"""

# On peut demander a gmsh de le mailler depuis le notebook

with open('A.geo', 'w') as f:
    f.write(geo_file)
    
coords, conn = meshGeo('A.geo')
plotMesh(coords, conn)